<a href="https://colab.research.google.com/github/Fatih120/GuildedChatExporter/blob/main/guildedchatexporter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuildedChatExporter v2

Export your stuff before it's gone.

Need help?
https://github.com/Fatih120/GuildedChatExporter/tree/main


[Mountain of Fatih Guilded](https://guilded.gg/MoF)

[Discord :(](https://discord.com/invite/Cy27FNfQtc)



In [ ]:
# @title Basic Setup

# @markdown 1.   Log on to Guilded on your web browser, not client, and focus on that tab.

# @markdown 2.   If based on **Firefox**, press `SHIFT + F9`.
# @markdown
# @markdown If on ***Chrome***, press `F12` and find the "Application" tab.
# @markdown
# @markdown We're here to grab your login, so click the Cookies entry on the sidebar, and then the guilded.gg url.
# @markdown
# @markdown Find the `hmac_signed_session` key at the bottom. Copy the value, it is very long. Paste it here.

import requests
import json
import re
import os
import ipywidgets as widgets
from concurrent.futures import ThreadPoolExecutor, as_completed

USER_TOKEN = "" # @param {type:"string"}
# @markdown Do NOT share this value with anyone as they will be able to access your account without a password.

SAVE_DIRECTORY = "/guildedchatexporter/" # @param {type:"string"}
if not os.path.exists(SAVE_DIRECTORY):
  os.makedirs(SAVE_DIRECTORY)

cookies = {
    "authenticated": "true",
    "hmac_signed_session": USER_TOKEN,
}

guilded = "https://www.guilded.gg/api"

# @markdown Google Colab gives you over 100GB of temporary space to work with, which should be way more than enough to store and then download your stuff.
# @markdown
# @markdown  You can also connect your Google Drive instead and copy the stuff to there at the end, though if you just want to download your files immediately without using Google Drive, skip the optional step.
# --------------------------------------------------- #

def fetch(endpoint, params=None):
    url = f"{guilded}/{endpoint}"
    response = requests.get(url, params=params, cookies=cookies)
    if response.status_code == 200:
      return unshid_cdn(response.json())
    else:
        raise Exception(f"Error fetching data: {response.status_code} {response.text}")
        print("Did you correctly input your token?")

def unshid_cdn(data):
  def fix_url(url):
    match = re.match(r"^https://s3-us-west-2\.amazonaws\.com/www\.guilded\.gg/(.*)$", url)
    if match:
      return f"https://cdn.gilcdn.com/{match.group(1)}"
    else:
      return url

  def fix_dict(d):
    for key, value in d.items():
      if isinstance(value, dict):
        fix_dict(value)
      elif isinstance(value, list):
        for item in value:
          if isinstance(item, dict):
            fix_dict(item)
          elif isinstance(item, str):
            d[key] = fix_url(item)
      elif isinstance(value, str):
        d[key] = fix_url(value)

  fix_dict(data)
  return data

def fetch_user(user_id):
    data = fetch(f"users/{user_id}")
    return data["user"]

def fetch_channel(channel_id):
    data = fetch("content/route/metadata", params={"route": f"//channels/{channel_id}/chat"})
    return data["metadata"]["channel"]

def fetch_servers():
    data = fetch("me", params={"isLogin": "false", "v2": "true"})
    return data['teams']

def fetch_members():
    data = fetch(f"teams/{SERVER_ID}/members")
    return data

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_groups():
    return fetch(f"teams/{SERVER_ID}/groups")

def get_channels():
    return fetch(f"teams/{SERVER_ID}/channels")

def fetch_myself():
    return fetch("me", params={"isLogin": "false", "v2": "true"})

def fetch_info():
    return fetch(f"teams/{SERVER_ID}/info")

def sanitize_filename(filename):
    # Replace invalid characters with underscores
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
    return filename[:255]


print("Please select the server you wish to archive in the dropdown.")
server_dict = {f"{server['name']} ({server['id']})": server["id"] for server in fetch_servers()}
server_dict[''] = None
server_dropdown = widgets.Dropdown(options=list(server_dict.keys()), value='', description='Servers:',)
display(server_dropdown)

def on_server_select(change):
    global SERVER_ID
    global SERVER_NAME
    selected_server_name = change["new"]
    SERVER_ID = server_dict[selected_server_name]
    SERVER_NAME = selected_server_name
    print(f"Selected server: {selected_server_name}")

server_dropdown.observe(on_server_select, names="value")

In [ ]:
# @title (Optional) Connect Google Drive
save_to_gdrive = True # @param {type:"boolean"}
from google.colab import drive
drive.mount('/content/drive')
if save_to_gdrive:
    SAVE_DIRECTORY = "/content/drive/MyDrive/guildedchatexporter/"
print("Done, do next step")

In [ ]:
# @title Save Basic Data about Server and Users
if 'SERVER_ID' not in globals():
    print("You forgot to choose a server. Go back and select one from the dropdown.")
else:
    server_dir = os.path.join(SAVE_DIRECTORY, SERVER_NAME)
    os.makedirs(server_dir, exist_ok=True)

verbose = True # @param {type:"boolean"}

def download_file(url, filepath):
    !wget -q -nc -O "$filepath" "$url"
mc = 0
members_data = fetch_members()
members = members_data.get('members', [])
for member in members:
    member_id = member["id"]
    member_name = member["name"]
    member_dir = os.path.join(server_dir, "members", member_name)
    os.makedirs(member_dir, exist_ok=True)
    user_data = fetch_user(member_id)

    with open(os.path.join(member_dir, f"{member_id}.json"), 'w') as f:
        f.write(json.dumps(user_data, indent=4))
    json_string = json.dumps(user_data)

    cdn_links = re.findall(r"https://cdn\.gilcdn\.com/[^\"]+", json_string)
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(download_file, url, os.path.join(member_dir, url.split("/")[-1].split("?")[0])) for url in cdn_links]

    mc += 1
    if verbose:
      print(f"{str(mc)}/" + str(len(members)) + f" {member_name} ({member_id})")

with open(os.path.join(server_dir, 'members.json'), 'w') as f:
    f.write(json.dumps(fetch_members(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/members.json")
with open(os.path.join(server_dir, 'info.json'), 'w') as f:
    f.write(json.dumps(fetch_info(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/info.json")
with open(os.path.join(server_dir, 'groups.json'), 'w') as f:
    f.write(json.dumps(get_groups(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/groups.json")
with open(os.path.join(server_dir, 'channels.json'), 'w') as f:
    f.write(json.dumps(get_channels(), indent=4))
    print(f"Saved {SAVE_DIRECTORY}{SERVER_NAME}/channels.json")

In [ ]:
# @title Save All Emoji
!pip install emoji


emotes_data = fetch(f"teams/{SERVER_ID}/customReactions")

verbose = True # @param {type:"boolean"}

def fetch_emotes():
    emotes_dir = os.path.join(server_dir, "emotes")
    os.makedirs(emotes_dir, exist_ok=True)

    with open(os.path.join(server_dir, 'emotes.json'), 'w') as f:
        f.write(json.dumps(emotes_data, indent=4))
    print("Saved emotes.json")

    ec = 0
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        downloaded = set()  # need to check if we already got it cuz dupes, waste less time
        for reaction in emotes_data.get('reactions', []):
            for image_type in ["png", "webp", "apng"]:
                url = reaction.get(image_type)
                if url and url not in downloaded:
                    filename = url.split("/")[-1].split("?")[0]
                    filepath = os.path.join(emotes_dir, reaction["name"] + "." + filename.split(".")[-1])
                    futures.append(executor.submit(download_emote, url, filepath))
                    downloaded.add(url)
                    ec += 1
                    if verbose:
                      print(f"{ec}/{str(len(emotes_data.get('reactions', [])))} " + reaction["name"])

        for future in as_completed(futures):
            future.result()
        print("All done, enjoy your emotes!")

def download_emote(url, filepath):
    !wget -q -nc -O "$filepath" "$url"

fetch_emotes()


In [ ]:
# @title HTML File Setup
# @markdown Has the base HTML and CSS information which you can edit. Just click RUN to set the info for the next step.
# @markdown
# @markdown This will have a flag in the future for if you don't want to download/show attachments, but why?
## HTML Template

CSS = """
@font-face {
  font-family: "Builder Sans";
  src: url("https://www.guilded.gg/fonts/BuilderSans-Regular.woff2") format('woff2');
}

body {
  font-family: 'Builder Sans', Tahoma, sans-serif;
  color: white;
  background-color: #373943;
}

#chat-log {
  background-color: #373943;
  padding: 16px;
  display: flex;
  flex-direction: column;
  gap: 8px;
}

.message {
  color: #ececee;
  font-size: 15px;
  line-height: 145%;
  font-weight: normal;
  font-family: 'Builder Sans', Tahoma, sans-serif;
  margin-top: 0px;
  min-height: 44px;
  padding: 8px 12px;
  display: flex;
  align-content: flex-start;
  gap: 8px;
  transition: background-color 140ms ease;
  position: relative;
  contain: layout;
  max-width: 80%;
}

.message:hover {
  background-color: #31333c;
}

.created-at {
  margin-right: 8px;
  height: 11px;
  color: #a3a3ac;
  font-size: 13px;
  line-height: 120%;
  font-weight: normal;
  white-space: nowrap;
  pointer-events: auto;
}

.created-by {
  font-size: 15px;
  line-height: 145%;
  font-weight: 700;
  white-space: nowrap;
  pointer-events: auto;
}

.pfp {
  width: 40px;
  height: 40px;
  border-radius: 50%;
  overflow: hidden;
  vertical-align: middle;
  flex-shrink: 0;
  margin-right: 12px;
  pointer-events: auto;
}

.content {
  margin: 0;
  word-break: break-all;
}

.content img,
.content video {
  max-height: 320px;
  max-width: 360px;
  object-fit: contain;
}

.file-upload-container {
  display: flex;
  align-items: center;
  background-color: #32343d;
  padding: 10px;
  border: 1px #a3a3ac solid;
  border-radius: 5px;
	color: #fef0b9;
}

.file-icon {
  margin-right: 10px;
	background-image: url('{icon-file-attach.svg');
  background-repeat: no-repeat;
  background-size: contain;
}

.file-info {
    flex-grow: 1;
}

.file-name {
    font-weight: bold;
    text-decoration: none;
    color: #fef0b9;
}

.file-size {
    color: #666;
    font-size: 0.9em;
}

.download-icon {
    margin-left: 10px;
    color: #666;
}

.icon {
    width: 20px;
    height: 20px;
}

"""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>#{title}</title>
    <style>
        {CSS}
    </style>
</head>
<body>
    <div id="chat-log">
        <p>#{title}</p>
        <p>{topic}</p>
        <hr>
        {messages}
    </div>
</body>
</html>
"""

## Message Template
MESSAGE_TEMPLATE = """
<div class="message">
    <img class="pfp" src="{avatar_url}">
    <div>
        <span class="created-by">{created_by}</span>
        <span class="created-at">({created_at})</span>
        <p class="content">{content}</p>
    </div>
</div>
"""

print("HTML ready to print")


In [ ]:
# @title Download all Media and Save HTMLs for every channel
verbose_mescount = True

from urllib.parse import urlparse, unquote
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import json
import requests
import emoji

# Load emotes data
with open(os.path.join(server_dir, "emotes.json"), "r") as f:
    emotes_data = json.load(f)
    emotes_dict = {str(emote['id']): emote for emote in emotes_data.get('reactions', [])}
# Load server info for roles
with open(os.path.join(server_dir, "info.json"), "r") as f:
    server_info = json.load(f)
    roles_iterable = server_info['team']['rolesById'].values() if isinstance(server_info['team']['rolesById'], dict) else server_info['team']['rolesById']
    roles_data = {str(role['id']): role for role in roles_iterable}
# Load members data
with open(os.path.join(server_dir, "members.json"), "r") as f:
    members_json = json.load(f)
    members_dict = {member['id']: member for member in members_json['members']}
# Load channels data
with open(os.path.join(server_dir, "channels.json"), "r") as f:
    channels_data = json.load(f)
with open(os.path.join(server_dir, "groups.json"), "r") as f:
    groups_data = json.load(f)

def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "wb") as f:
            f.write(response.content)

def generate_html(messages, channel_data, members_data, server_dir, channel_dir):
    formatted_messages = []
    doge = {}

    with ThreadPoolExecutor() as executor:
        futures = []

        for message in messages:
            created_by_id = message['createdBy']
            created_at = message.get('createdAt') or message.get('created_at')

            # Determine if it's a system message
            is_system_message = message.get('type') == 'system' or 'systemMessage' in str(message.get('content', {}))

            if is_system_message:
                created_by = "System Message"
                avatar_url = "https://www.guilded.gg/asset/DefaultUserAvatars/profile_5.png"

                # Handle system message content
                content = ""
                document = message.get('content', {}).get('document', {})
                for node in document.get('nodes', []):
                    if node.get('type') == 'systemMessage':
                        data = node.get('data', {})
                        message_type = data.get('type')
                        if message_type == 'team-channel-created':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user]')}) created this chat channel."
                        elif message_type == 'channel-renamed':
                            content = f"{created_by} ({data.get('createdBy', '[deleted user')}) renamed this channel from '{data.get('oldName')}' to '{data.get('newName')}'."
                        elif message_type == 'webhookMessage':
                            embeds = node.get('data', {}).get('embeds', [])
                            content = json.dumps(data, indent=2)
                        else:
                            content = f"System action performed: {message_type}"
            else:
                # Find member data based on createdBy ID
                member = next((m for m in members_data['members'] if m['id'] == created_by_id), None)
                created_by = member["name"] if member else "[deleted user]"

                # Get the highest priority role for the member
                highest_priority_role = None
                if member and 'roleIds' in member:
                    member_roles = [roles_data.get(str(role_id)) for role_id in member["roleIds"]]
                    member_roles = [role for role in member_roles if role is not None]
                    member_roles.sort(key=lambda x: x["priority"], reverse=True)
                    if member_roles:
                        highest_priority_role = member_roles[0]

                # Color the created_by name based on the highest priority role
                if highest_priority_role:
                    color1 = highest_priority_role["color"]
                    color2 = highest_priority_role.get("color2")
                    if color1 == "transparent":
                        # Find the next lower priority role with a valid color
                        next_role = None
                        for role in member_roles[1:]:
                            if role["color"] != "transparent":
                                next_role = role
                                break
                        if next_role:
                            color1 = next_role["color"]
                            color2 = next_role.get("color2")
                        else:
                            color1 = "#ffffff"  # Default to white if no valid color is found
                    if color2:
                        created_by = f'<span style="background: linear-gradient(to right, {color1}, {color2}); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{created_by}</span>'
                    else:
                        created_by = f'<span style="color: {color1};">{created_by}</span>'

                # Handle avatar URL
                if created_by_id in members_dict and members_dict[created_by_id].get('profilePicture'):
                    avatar_url = os.path.join("..", "..", "members", members_dict[created_by_id].get('name'), os.path.basename(members_dict[created_by_id].get('profilePicture')).split('?')[0])
                else:
                    if created_by_id not in doge:
                        avatar_number = (len(doge) % 5) + 1
                        doge[created_by_id] = avatar_number
                    avatar_url = f"https://www.guilded.gg/asset/DefaultUserAvatars/profile_{doge[created_by_id]}.png"

                # Process regular message content
                content = ""
                if "content" in message and message["content"]:
                    document = message["content"].get("document")
                    if document:
                        nodes = document.get("nodes", [])
                        for node in nodes:
                            if node.get("type") in ("image", "video", "fileUpload"):
                                file_src = node.get("data", {}).get("src", "")
                                if file_src:
                                    parsed_url = urlparse(file_src)
                                    media_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"

                                    if node.get("type") == "fileUpload":
                                        filename = node.get("data", {}).get("name")
                                        folder = "files"
                                    elif node.get("type") == "image":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "images"
                                    elif node.get("type") == "video":
                                        filename = os.path.basename(parsed_url.path)
                                        folder = "videos"

                                    filepath = os.path.join(channel_dir, "media", folder, filename)
                                    futures.append(executor.submit(download_file, media_url, filepath))

                                    if node.get("type") == "image":
                                        content += f'<img src="{os.path.join("media", "images", filename)}">'
                                    elif node.get("type") == "video":
                                        content += f'<video controls><source src="{os.path.join("media", "videos", filename)}" type="video/webm">Your browser does not support the video tag.</video>'
                                    elif node.get("type") == "fileUpload":
                                        file_size_bytes = node.get("data", {}).get("fileSizeBytes")
                                        file_size_kb = round(file_size_bytes / 1024, 1)
                                        content += f'''
                                            <div class="file-upload-container">
                                                <div class="file-icon">
                                                    <svg class="icon icon-file-attach" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-file-attach"></use>
                                                    </svg>
                                                </div>
                                                <div class="file-info">
                                                    <a href="{os.path.join("media", "files", filename)}" download class="file-name">{filename}</a>
                                                    <div class="file-size">{file_size_kb} kb</div>
                                                </div>
                                                <a href="{os.path.join("media", "files", filename)}" download class="download-icon">
                                                    <svg class="icon icon-download-tray" shape-rendering="geometricPrecision" role="img">
                                                        <use xml:space="http://www.w3.org/1999/xlink" xlink:href="#icon-download-tray"></use>
                                                    </svg>
                                                </a>
                                            </div>
                                        '''
                                    content += '<br>'

                            if node.get("type") == "paragraph":
                                sub_nodes = node.get("nodes", [])
                                for sub_node in sub_nodes:
                                    if sub_node.get("type") == "reaction":
                                        reaction_data = sub_node.get("data", {})
                                        if reaction_data:
                                            reaction = reaction_data.get("reaction")
                                            if reaction:
                                                reaction_id = reaction.get("id")
                                                if reaction_id:
                                                    if str(reaction_id).startswith("9000"):
                                                        reaction_name = reaction.get("name")
                                                        if reaction_name:
                                                            reaction_emoji = emoji.emojize(f":{reaction_name}:", language='alias')
                                                            content += reaction_emoji
                                                    else:
                                                        emoji_dict = next((item for item in emotes_dict.values() if item['id'] == reaction_id), None)
                                                        if emoji_dict:
                                                            content += f'<img src="{emoji_dict["webp"]}" alt="{emoji_dict["name"]}" title="{emoji_dict["name"]}" style="height: 1.3em; vertical-align: middle;">'
                                                        else:
                                                            custom_reaction = reaction.get("customReaction")
                                                            if isinstance(custom_reaction, bool):
                                                                custom_reaction = None
                                                            if custom_reaction and "webp" in custom_reaction:
                                                                emote_url = custom_reaction["webp"]
                                                                emote_filename = custom_reaction["name"]
                                                                emotes_external_dir = os.path.join(server_dir, "emotes", "external")
                                                                if not os.path.exists(emotes_external_dir):
                                                                    os.makedirs(emotes_external_dir)
                                                                emote_filepath = os.path.join(emotes_external_dir, emote_filename + ".webp")
                                                                if not os.path.exists(emote_filepath):
                                                                    response = requests.get(emote_url)
                                                                    if response.status_code == 200:
                                                                        with open(emote_filepath, "wb") as f:
                                                                            f.write(response.content)
                                                                content += f'<img src="../../emotes/external/{emote_filename}.webp" alt="{emote_filename}.webp" title="{emote_filename}.webp" style="height: 1.3em; vertical-align: middle;">'
                                    else:
                                        leaves = sub_node.get("leaves", [])
                                        for leaf in leaves:
                                            leaf_text = leaf.get("text", "")
                                            marks = leaf.get("marks", [])
                                            for mark in marks:
                                                mark_type = mark.get("type")
                                                if mark_type == "inline-code-v2":
                                                    leaf_text = f'<code>{leaf_text}</code>'
                                                elif mark_type == "bold":
                                                    leaf_text = f'<strong>{leaf_text}</strong>'
                                                elif mark_type == "italic":
                                                    leaf_text = f'<em>{leaf_text}</em>'
                                                elif mark_type == "strikethrough":
                                                    leaf_text = f'<del>{leaf_text}</del>'
                                            content += leaf_text
                                content += '<br>'

            formatted_message = MESSAGE_TEMPLATE.format(
                avatar_url=avatar_url,
                created_by=created_by,
                created_at=created_at,
                content=content
            )
            formatted_messages.append(formatted_message)

        # Wait for all download tasks to complete
        for future in as_completed(futures):
            future.result()

    html = HTML_TEMPLATE.format(
        title=channel_data["name"],
        topic=channel_data.get("topic", ""),
        messages='\n'.join(formatted_messages),
        CSS=CSS
    )

    return html



# Process each channel
for channel in channels_data["channels"]:
    channel_dir = os.path.join(server_dir, channel["name"])
    for group in groups_data["groups"]:
        if channel["groupId"] == group["id"]:
            print(f"Processing {group['name']} #{channel['name']}")
            group_dir = os.path.join(server_dir, group["name"] + " (" + group["id"] + ")")
            channel_dir = os.path.join(group_dir, channel["name"])
            break
    os.makedirs(channel_dir, exist_ok=True)

    # Create media folders
    os.makedirs(os.path.join(channel_dir, "media", "images"), exist_ok=True)
    os.makedirs(os.path.join(channel_dir, "media", "videos"), exist_ok=True)
    os.makedirs(os.path.join(channel_dir, "media", "files"), exist_ok=True)

    # Save channel info
    with open(os.path.join(channel_dir, f"{channel['id']}_info.json"), 'w') as f:
        json.dump(channel, f, indent=4)

    # Check if the channel's content type is not "chat", "stream", or "voice"
    if channel.get("contentType") in ["chat", "stream", "voice"]:
        print(f"=========== FETCHING MESSAGES FROM #{channel['name']}...")
        messages = []
        beforeDate = None
        mesc = 0

        while True:
            params = {"limit": 100}
            if beforeDate:
                params["beforeDate"] = beforeDate

            response = fetch(f"channels/{channel['id']}/messages", params=params)
            messages.extend(response["messages"])

            mesc += len(response["messages"])
            #print(f"Collected {mesc} messages")

            if len(response["messages"]) < 100:
                break

            beforeDate = response["messages"][-1]["createdAt"]

        messages.reverse()

        # Save messages JSON
        with open(os.path.join(channel_dir, f"{channel['id']}_messages.json"), 'w') as f:
            json.dump(messages, f, indent=4)

        if verbose_mescount:
            print(f"Found {len(messages)} messages")
        print(f"Saved all messages from #{channel['name']} to {channel_dir}/{channel['id']}_messages.json")

        # Generate and save HTML
        html = generate_html(messages, channel, members_json, server_dir, channel_dir)
        html_filename = f"{channel['name']}.html".replace('/', '-')
        with open(os.path.join(channel_dir, html_filename), "w", encoding='utf-8') as f:
            f.write(html)

        print(f"Saved HTML to {channel_dir}/{['name']}.html")
    else:
        print(f"#{channel['name']} is {channel['contentType']}, skipping rn")

print("Finished processing all channels.")

In [ ]:
# @title preview html (wont show local files)
from IPython.display import HTML
with open(os.path.join(server_dir, f"{channel_data['name']}.html"), "r") as f:
    html_content = f.read()
display(HTML(html_content))


In [ ]:
# @title Download the whole server free no virus
import shutil
from google.colab import files
shutil.make_archive(SERVER_NAME, 'zip', SAVE_DIRECTORY, SERVER_NAME)
files.download(SERVER_NAME + '.zip')

if save_to_gdrive:
    shutil.copy(SERVER_NAME + '.zip', '/content/drive/MyDrive/' + SERVER_NAME + '.zip')
    print("Copied zip to Google Drive.")


In [ ]:
myself_data = fetch_myself()
with open(os.path.join(SAVE_DIRECTORY, "myself.json"), "w") as f:
    f.write(json.dumps(myself_data, indent=4))
    print(f"Saved {SAVE_DIRECTORY}myself.json")
USER_ID = myself_data["user"]["id"]
def get_groups():
    return fetch(f"users/{USER_ID}/channels")

def get_dm_ids(data):
    if "channels" in data and isinstance(data["channels"], list) and len(data["channels"]) > 0:
        return data["channels"][0]["id"]
    return None

channel_id = get_dm_ids(myself_data)
print(f"{channel_id}")